# Profiling TensorFlow Single GPU Single Node Training Job with Amazon SageMaker Debugger

***Note: 본 노트북 코드는 [영문 노트북](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-debugger/tensorflow_profiling/tf-resnet-profiling-single-gpu-single-node.ipynb)을 한국어화(중간중간 역주 추가와 documentation 보완)하면서 보완하면서, Debugger 예제 코드와 TensorBoard 로깅 코드를 추가하였습니다.***

이 노트북은 SageMaker 디버거 프로파일링(debugger profiling) 기능이 활성화된 상태에서 TensorFlow 훈련 작업을 생성하는 과정을 안내하며, 단일 노드 내의 단일 GPU 훈련을 수행합니다.

### Install SageMaker and smdebug
2020년 12월에 출시된 새로운 디버거 프로파일링 기능을 사용하려면, 최신 버전의 SageMaker 및 SMDebug SDK가 설치되어 있는지 확인하세요. 다음 코드 셀을 사용하여 라이브러리를 업데이트하고 Jupyter 커널을 다시 시작하여 업데이트를 적용합니다.

In [ ]:
import sys
import IPython
import boto3    
import sagemaker    
install_needed = False  # should only be True once
if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U sagemaker smdebug
    IPython.Application.instance().kernel.do_shutdown(True)

bucket = sagemaker.Session().default_bucket()    

<br>

## 1. Create a Training Job with Profiling Enabled<a class="anchor" id="option-1"></a>
---

[SageMaker Estimator API for Tensorflow](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator)를 사용하여 훈련 작업을 생성합니다. 프로파일링을 활성화하려면 `ProfilerConfig` 객체를 생성하고 TensorFlow estimator의 `profiler_config` 파라메터에 전달합니다.

### Define hyperparameters

Epoch 수, 배치 크기 및 데이터 증강(augmentation)와 같은 하이퍼파라메터를 정의합니다. 시스템 사용률을 높이기 위해 배치 크기를 늘릴 수 있지만 CPU 병목 문제가 발생할 수 있습니다. 증강을 통한 대규모 배치 크기의 데이터 전처리에는 많은 계산이 필요합니다. data_augmentation을 비활성화하여 시스템 활용도에 미치는 영향을 확인할 수 있습니다.

데모 목적으로 CPU 사용률을 늘리기 위해 다음과 같은 하이퍼파라미터를 준비합니다.

In [ ]:
hyperparameters = {"epoch": 2, "batch_size": 64, "data_augmentation": True}

### Configure rules

다음 규칙(rule)들을 지정합니다. 

- `loss_not_decreasing` : loss가 감소하고 있는지 확인하고 지난 몇 번의 iteration에서 loss이 특정 수치만큼 감소하지 않은 경우 트리거합니다.
- `vanishing_gradient` : graidnet가 매우 작아지거나 0으로 떨어지는지를 감지합니다. gradient의 절대값 평균이 지정된 임계값 미만으로 감소하는 경우 트리거합니다.
- `LowGPUUtilization` : GPU의 활용도가 낮은지 확인합니다.
- `ProfilerReport` : 전체 퍼포먼스 규칙 세트를 실행하고 추가 정보 및 권장 사항이 포함된 최종 출력용 보고서를 생성합니다.

In [ ]:
from sagemaker.debugger import (
    Rule, 
    ProfilerRule, 
    DebuggerHookConfig,
    TensorBoardOutputConfig,
    CollectionConfig,    
    rule_configs
)    

rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),    
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport())
]

### Configure DebuggerHookConfig and TensorBoard

DebuggerHookConfig 및 TensorBoardOutputConfig 클래스 인스턴스를 Estimator에 전달 시, SageMaker Debugger는 TensorBoard와 호환되는 출력 텐서 파일을 훈련 인스턴스의 `/opt/ml/output/tensor` 경로에 자동으로 생성합니다. 훈련이 종료되면 각각 S3의 `debug-output`, `tensorboard-output` 폴더로 복사되며 이를 복사하여 로컬 상에서 텐서보드를 띄울 수도 있습니다.

In [ ]:
hook_config = DebuggerHookConfig(
    hook_parameters={"save_interval": "100"},
    collection_configs=[
        CollectionConfig("weights"),
        CollectionConfig(
            name="biases", 
            parameters={"save_interval": "10", "end_step": "500", "save_histogram": "True"}
        ),
    ],
)

tb_config = TensorBoardOutputConfig(
    s3_output_path='s3://{}/tensorboard-resnet'.format(bucket)
)

### Specify a profiler configuration

다음 설정은 500 밀리초(0.5초)에서 시스템 메트릭을 캡처합니다. 시스템 지표에는 CPU 당 사용률, GPU, CPU 당 메모리 사용률, GPU, I/O 및 네트워크가 포함됩니다.

디버거는 5단계부터 15단계까지 상세한 프로파일링 정보를 캡처합니다. 이 정보에는 Horovod 메트릭, 데이터 로딩, 전처리, CPU 및 GPU에서 실행되는 연산자(operator)가 포함됩니다.

In [ ]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(
        local_path="/opt/ml/output/profiler/", start_step=5, num_steps=10
    ),
)

### Get the image URI

이 노트북을 실행하는 리전에 따라 도커 이미지가 달라집니다.

In [ ]:
session = boto3.session.Session()
region = session.region_name
image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/tensorflow-training:2.4.1-gpu-py37-cu110-ubuntu18.04"

### Define estimator

프로파일링을 활성화하려면 디버거 프로파일링 구성 (`profiler_config`), 디버거 규칙 목록 (`rules`) 및 이미지 URI (`image_uri)`를 estimator에 전달해야 합니다. 디버거는 SageMaker estimator가 훈련 작업을 요청하는 동안 모니터링 및 프로파일링을 활성화합니다.

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    role=sagemaker.get_execution_role(),
    image_uri=image_uri,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    entry_point="train_tf.py",
    source_dir="entry_point",
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,    
    debugger_hook_config=hook_config,    
    tensorboard_output_config=tb_config,
    rules=rules
)

### Start training job

`wait=False` argument를 포함한 `estimator.fit()`은 백그라운드에서 훈련 작업을 시작합니다. 대시보드 또는 분석 노트북 실행을 계속할 수 있습니다.

In [ ]:
estimator.fit(wait=False)

In [ ]:
estimator.latest_training_job.rule_job_summary()

<br>

## 2. Analyze Profiling Data
---

다음 셀 (`training_job_name` 및 `region`)의 출력을 복사하여 분석 노트북 `profiling_generic_dashboard.ipynb`, `analyze_performance_bottlenecks.ipynb` 및 `profiling_interactive_analysis.ipynb`를 실행합니다.

In [ ]:
training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

훈련이 아직 진행 중인 동안 SageMaker Studio 또는 노트북에서 성능 데이터를 시각화할 수 있습니다. 디버거는 타임라인 차트 또는 히트맵 형식으로 시스템 메트릭을 그리는 유틸리티를 제공합니다. 자세한 내용은 노트북 [profiling_interactive_analysis.ipynb](analysis_tools/profiling_interactive_analysis.ipynb)를 확인하세요. 다음 코드 셀에서 총 CPU 및 GPU 사용률을 시계열 차트로 표시합니다. I/O, 메모리, 네트워크와 같은 다른 메트릭을 시각화하려면, `select_dimension` 및 `select_events`에 전달된 목록을 확장하기만 하면 됩니다.

### Access the profiling data using the SMDebug `TrainingJob` utility class
 
Profiler 데이터가 준비될 때까지 기다립니다. Profiler 데이터는 훈련 인스턴스 프로비저닝-훈련 데이터 다운로드-훈련 시작 이후 일정 iteration 이상 반복해야 수집되기에 몇 분의 시간이 소요됩니다.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

Debugger Rule에서 정의한 Rule에 대한 CloudWatch 로드 URL을 생성합니다. URL로 접속해서 로그를 직접 확인해 보세요.

In [ ]:
def _get_rule_job_name(training_job_name, rule_configuration_name, rule_job_arn):
    """Helper function to get the rule job name with correct casing"""
    return "{}-{}-{}".format(
        training_job_name[:26], rule_configuration_name[:26], rule_job_arn[-8:]
    )


def _get_cw_url_for_rule_job(rule_job_name, region):
    return "https://{}.console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix".format(
        region, region, rule_job_name
    )


def get_rule_jobs_cw_urls(estimator):
    training_job = estimator.latest_training_job
    training_job_name = training_job.describe()["TrainingJobName"]
    rule_eval_statuses = training_job.describe()["DebugRuleEvaluationStatuses"]

    result = {}
    for status in rule_eval_statuses:
        if status.get("RuleEvaluationJobArn", None) is not None:
            rule_job_name = _get_rule_job_name(
                training_job_name, status["RuleConfigurationName"], status["RuleEvaluationJobArn"]
            )
            result[status["RuleConfigurationName"]] = _get_cw_url_for_rule_job(
                rule_job_name, boto3.Session().region_name
            )
    return result


get_rule_jobs_cw_urls(estimator)

`wait=False`로 설정한 경우 아래 코드 셀을 실행하여 Synchronous 방식으로 변경할 수도 있습니다.

In [ ]:
#sagemaker.Session().logs_for_job(job_name=estimator.latest_training_job.job_name, wait=True)

<br>

## 3. Download Debugger Profiling Report and Plotting Metrics
---

`ProfilerReport()` 규칙은 기본 규칙 요약 및 다음 단계 권장 사항이 포함된 html 리포트 `profiler-report.html`을 생성합니다. 이 리포트는 S3 버킷에서 찾을 수 있습니다.

In [ ]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

아래 코드 셀을 실행시키면 프로파일링 리포트를 로컬 환경으로 다운로드하고 리포트를 다이렉트로 확인할 수 있는 링크를 생성합니다. 
링크를 새 탭이나 새 창으로 연 다음 프로파일링 보고서를 확인하세요

디버거 프로파일링 보고서를 다운로드하고 여는 방법에 대한 자세한 내용은 SageMaker 개발자 가이드의 [SageMaker Debugger Profiling Report](https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-profiling-report.html)를 참조하세요.

**[주의] 만약 JupyterLab에서 실행하는 경우 화면 좌측 상단의 Trust HTML을 클릭해야 html 리포트가 정상적으로 출력됩니다!**

In [ ]:
output_dir = './output'
profile_output = output_dir+'/ProfilerReport'

!rm -rf $output_dir

import json, os
from IPython.core.display import display, HTML

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
if not os.path.exists(profile_output):
    os.makedirs(profile_output)    
    
!aws s3 ls {rule_output_path}/ProfilerReport/profiler-output/
!aws s3 cp {rule_output_path}/ProfilerReport/profiler-output/ {output_dir}/ProfilerReport/ --recursive    

display(HTML('<b>ProfilerReport : <a href="{}profiler-report.html">Profiler Report</a></b>'.format(output_dir+"/ProfilerReport/")))

In [ ]:
training_job_name = estimator.latest_training_job.job_name
region = sagemaker.Session().boto_region_name

tj = TrainingJob(estimator.latest_training_job.job_name, sagemaker.Session().boto_region_name)

# Retrieve a description of the training job description and the S3 bucket URI where the metric data are saved
tj.describe_training_job()
tj.get_config_and_profiler_s3_output_path()

In [ ]:
# Wait for the data to be available
tj.wait_for_sys_profiling_data_to_be_available()
tj.wait_for_framework_profiling_data_to_be_available()

# Get the metrics reader
system_metrics_reader = tj.get_systems_metrics_reader()
framework_metrics_reader = tj.get_framework_metrics_reader()

# Refresh the event file list
system_metrics_reader.refresh_event_file_list()
framework_metrics_reader.refresh_event_file_list()

### Plotting Metrics

SageMaker Debugger Profiler는 시스템 지표와 프레임워크 지표를 쉽게 확인할 수 있는 5가지 클래스들을 제공하고 있습니다. 
- MetricsHistogram
- StepTimelineChart
- StepHistogram
- TimelineCharts
- Heatmap
 
`select_dimensions`와 `select_events` argument는 optional 파라메터로, 이를 지정하여 필요한 지표만 시각화할 수 있습니다. 예를 들어, `select_dimensions = ["GPU"]`를 지정하면 plot 메서드는 "GPU" 키워드를 포함하는 지표를 필터링합니다. 유사하게 `select_events = ["total"]`을 지정하면 "total" 이벤트 태그를 포함하는 지표를 필터링합니다.
자세한 내용은 https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-access-data-profiling-default-plot.html 를 참조하세요.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.metrics_histogram import MetricsHistogram
from smdebug.profiler.analysis.notebook_utils.step_timeline_chart import StepTimelineChart
from smdebug.profiler.analysis.notebook_utils.step_histogram import StepHistogram
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
from smdebug.profiler.analysis.notebook_utils.heatmap import Heatmap
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

#### System metrics histogram

In [ ]:
metrics_histogram = MetricsHistogram(system_metrics_reader)
metrics_histogram.plot(
    starttime=0, 
    endtime=system_metrics_reader.get_timestamp_of_latest_available_file(), 
    select_dimensions=["CPU", "GPU", "I/O"],
    select_events=["total"]
)

#### System and framework metrics heatmap

In [ ]:
view_heatmap = Heatmap(
    system_metrics_reader,
    framework_metrics_reader,
    select_dimensions=["CPU", "GPU", "I/O"],
    select_events=["total"],
    plot_height=200
)

#### Time line charts

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

view_timeline_charts = TimelineCharts(
    system_metrics_reader, 
    framework_metrics_reader,
    select_dimensions=["CPU", "GPU", "I/O"], # optional
    select_events=["total"]                  # optional 
)